# Scraping Script with parallelization

In [1]:
import pandas as pd
import time
import re
import requests
from bs4 import BeautifulSoup
import random
import numpy as np
import multiprocessing
import time

In [2]:
#Loading hole carl list
db_11912 = pd.read_csv('autocasion_unique.csv')

In [3]:
#Cleaning duplicated links, to brin only unique links
no_rep = db_11912['7'].value_counts()
no_rep

/coches-segunda-mano/nissan-micra-ocasion/micra-ig-t-s-s-acenta-90-ref6794986                        6
/coches-segunda-mano/renault-clio-ocasion/clio-1-5dci-eco2-s-s-energy-business-90-ref7093817         6
/coches-segunda-mano/kia-cee-d-ocasion/cee-d-1-4crdi-wgt-business-90-ref6896415                      6
/coches-segunda-mano/renault-clio-ocasion/clio-1-5dci-eco2-energy-authentique-75-ref7106771          6
/coches-segunda-mano/opel-corsa-ocasion/corsa-1-4-selective-90-1-ref6871315                          6
                                                                                                    ..
/coches-segunda-mano/renault-clio-ocasion/business-energy-dci-55kw-75cv-1623e678ddd8dd-ref6642850    1
/coches-segunda-mano/skoda-fabia-ocasion/1-0-tsi-70kw-95cv-ambition-161f3fcbc8cc67-ref6387154        1
/coches-segunda-mano/dacia-duster-ocasion/duster-1-6-ambiance-4x2-115-ref6620896                     1
/coches-segunda-mano/opel-zafira-ocasion/zafira-tourer-2-0cdti-expression

In [4]:
#creating list of index/links to scrap
no_rep_df = pd.DataFrame(no_rep)
index = no_rep_df.index
links_no_rep_list = []
for link in index:
    links_no_rep_list.append(link)


In [5]:
#Spliting list of  in equal qty of lists
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [6]:
#Dividing list into 8
div = chunkIt(links_no_rep_list, 8)

In [7]:
#List o list splited 
list1 = div[0]
list2 = div[1]
list3 = div[2]
list4 = div[3]
list5 = div[4]
list6 = div[5]
list7= div[6]
list8 = div[7]


In [8]:
#dummy lists for test

In [9]:
list1a = ['/coches-segunda-mano/nissan-micra-ocasion/micra-ig-t-s-s-acenta-90-ref6794986',
 '/coches-segunda-mano/kia-cee-d-ocasion/cee-d-1-4crdi-wgt-business-90-ref6896415',
 '/coches-segunda-mano/renault-clio-ocasion/clio-1-5dci-eco2-energy-authentique-75-ref7106771',
 '/coches-segunda-mano/opel-corsa-ocasion/corsa-1-4-selective-90-1-ref6871315']

In [10]:
list1b =['/coches-segunda-mano/opel-insignia-ocasion/insignia-1-6cdti-ecoflex-s-s-selective-136-ref7178412',
 '/coches-segunda-mano/dacia-sandero-ocasion/sandero-0-9-tce-stepway-66kw-90-5p-20-ref7208696',
 '/coches-segunda-mano/fiat-punto-ocasion/1-2-8v-easy-51kw-69cv-s-s-gasolina-162e35cfbdb87f-ref7206038',
 '/coches-km0/km-0/fiat-500-km0/500-1-0-hybrid-cult-52kw-ref5721940']

Main scraping code

In [12]:
"""def scraping_script(list_):
  print('doing something that takes ' + str(list_) + ' seconds')
  time.sleep(list_)
  print('Task done')"""


"def scraping_script(list_):\n  print('doing something that takes ' + str(list_) + ' seconds')\n  time.sleep(list_)\n  print('Task done')"

In [13]:
"""
start = time.perf_counter()

scraping_script(2)
scraping_script(2)

finish = time.perf_counter()

print('finished in ' + str(finish - start) + ' seconds')"""

"\nstart = time.perf_counter()\n\nscraping_script(2)\nscraping_script(2)\n\nfinish = time.perf_counter()\n\nprint('finished in ' + str(finish - start) + ' seconds')"

In [52]:
p1 = multiprocessing.Process(target = scraping_script, args = [2])
p2 = multiprocessing.Process(target = scraping_script, args = [3])

p1.start()
p2.start()


finish = time.perf_counter()
print('finished in ' + str(finish - start) + ' seconds')

finished in 5145.8145745 seconds


In [11]:
def scraping_script(list_):
    #print('Started '+ ' ........')
    count = 0
    main_list_ind_with_link =[]
    
    for element in list_:
        url = 'https://www.autocasion.com/'+element
        #print(url)
        time.sleep(random.randint(0,5))
        headers= {'Accept-Encoding':'gzip, deflate',
              'Accept-Language':'en-US,en;q=0.9',
              'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}
        response = requests.get(url ,headers=headers)
        soup = BeautifulSoup(response.content)

        maker = soup.find_all('div',{'class':'bloque titulo-ficha'})[0].find_all_next(text=True)[1].strip().split()[0]#Maker
        model = soup.find_all('div',{'class':'bloque titulo-ficha'})[0].find_all_next(text=True)[1].replace(maker,'').strip()
        descrip = soup.find_all('div',{'class':'bloque titulo-ficha'})[0].find_all_next(text=True)[2].strip()
        model_comp = soup.find_all('div',{'class':'bloque titulo-ficha'})[0].h1.text.strip() #Model
        year= soup.find_all('div',{'class':'bloque titulo-ficha'})[0].p.span.text.strip() #year
        city = soup.find_all('div',{'class':'bloque titulo-ficha'})[0].p.select('span')[1].text.strip() #city 
        year2 =soup.find_all('ul',{'class':'datos-basicos-ficha'})[0].find_all('li')[0].select('span')[1].text.strip() #year
        color = soup.find_all('ul',{'class':'datos-basicos-ficha'})[0].find_all('li')[7].select('span')[1].text.strip()#color
        power = soup.find_all('ul',{'class':'datos-basicos-ficha'})[0].find_all('li')[5].select('span')[1].text.strip() #Power
        motor = soup.find_all('ul',{'class':'datos-basicos-ficha'})[0].find_all('li')[4].select('span')[1].text.strip() #automanual
        chasis = soup.find_all('ul',{'class':'datos-basicos-ficha'})[0].find_all('li')[3].select('span')[1].text.strip() #chasis
        kilom = soup.find_all('ul',{'class':'datos-basicos-ficha'})[0].find_all('li')[2].select('span')[1].text.strip() #kilometer
        fuel = soup.find_all('ul',{'class':'datos-basicos-ficha'})[0].find_all('li')[1].select('span')[1].text.strip() #fuel
        url_car = element

        specs_car_list = []
        specs_car_list.append(maker)
        specs_car_list.append(model)
        specs_car_list.append(descrip)
        specs_car_list.append(model_comp)
        specs_car_list.append(year)
        specs_car_list.append(city)
        specs_car_list.append(year2)
        specs_car_list.append(color)
        specs_car_list.append(power)
        specs_car_list.append(motor)
        specs_car_list.append(chasis)
        specs_car_list.append(kilom)
        specs_car_list.append(fuel)
        specs_car_list.append(element)

        #print(specs_car_list)
        main_list_ind_with_link.append(specs_car_list)
        count = count+1
        print(count,' ',model," ",year)
        print(url)
    num = np.random.randint(1,10000000)
    main_dfindlink = pd.DataFrame(main_list_ind_with_link, columns =['Maker','Model','Description','Name','Year','City','Year2','Color','Power','Motor','Chasis','Kilometers','Fuel','Link'])
    main_dfindlink.to_csv(f'autocasion_parallelization_test{num}.csv')
    print('Task done')
    return main_dfindlink

In [ ]:
from multiprocessing import Pool

p = Pool()
p.map(scraping_script,['/coches-segunda-mano/nissan-micra-ocasion/micra-ig-t-s-s-acenta-90-ref6794986'])
p.terminate()
p.join()
#p1 = multiprocessing.Process(target = scraping_script, args = [list1a])
#p2 = multiprocessing.Process(target = scraping_script, args = [list1b])


#p1.run()
#p2.run()
#join

#p1.join()
#p2.join()


print('finished')

In [68]:
scraping_script(list1a)

1   Micra   2018
https://www.autocasion.com//coches-segunda-mano/nissan-micra-ocasion/micra-ig-t-s-s-acenta-90-ref6794986
2   Cee´d   2017
https://www.autocasion.com//coches-segunda-mano/kia-cee-d-ocasion/cee-d-1-4crdi-wgt-business-90-ref6896415
3   Clio   2014
https://www.autocasion.com//coches-segunda-mano/renault-clio-ocasion/clio-1-5dci-eco2-energy-authentique-75-ref7106771
4   Corsa   2018
https://www.autocasion.com//coches-segunda-mano/opel-corsa-ocasion/corsa-1-4-selective-90-1-ref6871315
Task done
